In [68]:

from groundingdino.util.inference import load_model, load_image, predict, annotate
import cv2

model = load_model("groundingdino/config/GroundingDINO_SwinT_OGC.py", "/home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth")
IMAGE_PATH = "/home/pearson/Projects/ECE253/Image_process/example_jpg/output_CLAHE/output_clahe_IMG_3580.jpeg"
TEXT_PROMPT = "person . car ."
BOX_TRESHOLD = 0.35
TEXT_TRESHOLD = 0.25

image_source, image = load_image(IMAGE_PATH)

boxes, logits, phrases = predict(
    model=model,
    image=image,
    caption=TEXT_PROMPT,
    box_threshold=BOX_TRESHOLD,
    text_threshold=TEXT_TRESHOLD,
    device="cpu"
)

annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
cv2.imwrite("output_clahe_IMG_3580_annoted.jpg", annotated_frame)

num_boxes = len(boxes)

if num_boxes > 0:
    # 2. 计算平均 Score (logits 是 Tensor，使用 .mean() 计算平均值，.item() 转为 Python float)
    avg_score = logits.mean().item()
    
    print(f"检测到的框数量: {num_boxes}")
    print(f"平均置信度 (Average Score): {avg_score:.4f}")
    
    # 如果你想看最高和最低分：
    print(f"最高 Score: {logits.max().item():.4f}")
    print(f"最低 Score: {logits.min().item():.4f}")
else:
    print("未检测到任何目标。")

final text_encoder_type: bert-base-uncased


检测到的框数量: 4
平均置信度 (Average Score): 0.6700
最高 Score: 0.8263
最低 Score: 0.5957


In [ ]:
import os
import cv2
import torch
import glob
import numpy as np  
from tqdm import tqdm
from groundingdino.util.inference import load_model, load_image, predict, annotate

def process_single_folder(model, input_dir, output_dir, config):
    """
    处理单个文件夹，并计算详细统计数据。
    """
    # 解包配置
    text_prompt = config["text_prompt"]
    box_threshold = config["box_threshold"]
    text_threshold = config["text_threshold"]
    device = config["device"]

    os.makedirs(output_dir, exist_ok=True)
    
    # 获取图片列表
    image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.PNG', '*.JPG', '*.JPEG', '*.BMP']
    image_paths = []
    for ext in image_extensions:
        image_paths.extend(glob.glob(os.path.join(input_dir, ext)))
    
    print(f"\n{'-'*60}")
    print(f"Processing Folder: {input_dir}")
    print(f"Output Directory:  {output_dir}")
    print(f"Images Found:      {len(image_paths)}")
    print(f"{'-'*60}")

    if not image_paths:
        print("No images found. Skipping.")
        return

    # --- 数据容器 ---
    # 1. 存储该文件夹下所有图片的所有 Score，用于计算文件夹级别的 Mean 和 Variance
    all_scores_in_folder = []
    # 2. 存储每张图片的详细信息 (如果你后续需要导出到Excel，可以用这个列表)
    per_image_stats = []
    # 3. 文件夹总框数计数器
    folder_total_boxes = 0

    # --- 循环处理图片 ---
    for img_path in tqdm(image_paths, desc="Progress"):
        filename = os.path.basename(img_path)
        
        try:
            image_source, image = load_image(img_path)

            boxes, logits, phrases = predict(
                model=model,
                image=image,
                caption=text_prompt,
                box_threshold=box_threshold,
                text_threshold=text_threshold,
                device=device
            )

            # 保存可视化结果
            annotated_frame = annotate(image_source=image_source, boxes=boxes, logits=logits, phrases=phrases)
            cv2.imwrite(os.path.join(output_dir, f"annotated_{filename}"), annotated_frame)

            # --- 收集单张图片统计信息 ---
            current_count = len(boxes)
            # 将 Tensor 转为 list
            current_scores = logits.tolist() 

            # 存储单张图片数据
            per_image_stats.append({
                "filename": filename,
                "box_count": current_count,
                "scores": current_scores
            })

            # 更新文件夹级别的汇总数据
            folder_total_boxes += current_count
            all_scores_in_folder.extend(current_scores)

        except Exception as e:
            print(f"Error processing {filename}: {e}")

    # --- 计算文件夹级别的统计 ---
    print(f"\n>>> STATS REPORT FOR FOLDER: {os.path.basename(input_dir)}")
    
    # 1. 总框数
    print(f"Total Boxes Detected: {folder_total_boxes}")

    # 2. 平均分和方差 (需要判断列表是否为空)
    if len(all_scores_in_folder) > 0:
        # 使用 numpy 计算平均值和方差
        folder_mean = np.mean(all_scores_in_folder)
        folder_variance = np.var(all_scores_in_folder)
        
        print(f"Average Confidence:   {folder_mean:.4f}")
        print(f"Confidence Variance:  {folder_variance:.4f}")
    else:
        print("Average Confidence:   N/A (No detections)")
        print("Confidence Variance:  N/A")
    
    # (可选) 打印每张图片的详情
    # for stat in per_image_stats:
    #     print(f"  File: {stat['filename']}, Count: {stat['box_count']}, Scores: {stat['scores']}")

    print(f"{'='*60}\n")


def main():
    # --- 1. 配置区域 (Configuration) ---
    # 将所有变量定义在这里，不使用全局变量
    cfg = {
        "config_path": "groundingdino/config/GroundingDINO_SwinT_OGC.py",
        "weights_path": "/home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth",
        "text_prompt": "person . car .",
        "box_threshold": 0.35,
        "text_threshold": 0.25,
        "device": "cpu"
    }

    # 定义要处理的文件夹列表 [(Input, Output), (Input, Output)]
    folders_to_process = [
        (
            "../dataset", 
            "../results/annoted/original/"
        ),
        (
            "../results/CLAHE/", 
            "../results/annoted/CLAHE/"
        ),
        (
            "../results/CLAHE_Wiener/", 
            "../results/annoted/CLAHE_Wiener/"
        ),
        (
            "../results/CLAHE_NAFNet/", 
            "../results/annoted/CLAHE_NAFNet/"
        ),
        (
            "../results/ZeroDCE/", 
            "../results/annoted/ZeroDCE/"
        ),
        (
            "../results/ZeroDCE_Wiener/", 
            "../results/annoted/ZeroDCE_Wiener/"
        ),
        (
            "../results/ZeroDCE_NAFNet/", 
            "../results/annoted/ZeroDCE_NAFNet/"
        ),
        # 在这里添加第二个文件夹...
        # ("/path/to/input2", "/path/to/output2"),
    ]

    # --- 2. 加载模型 (只加载一次) ---
    print(f"Loading Model from {cfg['weights_path']} ...")
    model = load_model(cfg["config_path"], cfg["weights_path"])

    # --- 3. 执行批处理 ---
    for input_dir, output_dir in folders_to_process:
        process_single_folder(model, input_dir, output_dir, cfg)

    print("All batch processing complete.")

if __name__ == "__main__":
    main()

Loading Model from /home/pearson/Projects/ECE253/GroundingDINO/weights/groundingdino_swint_ogc.pth ...
final text_encoder_type: bert-base-uncased

------------------------------------------------------------
Processing Folder: ../dataset
Output Directory:  ../results/annoted/original/
Images Found:      0
------------------------------------------------------------
No images found. Skipping.

------------------------------------------------------------
Processing Folder: ../results/CLAHE/
Output Directory:  ../results/annoted/CLAHE/
Images Found:      0
------------------------------------------------------------
No images found. Skipping.

------------------------------------------------------------
Processing Folder: ../results/CLAHE_Wiener/
Output Directory:  ../results/annoted/CLAHE_Wiener/
Images Found:      0
------------------------------------------------------------
No images found. Skipping.

------------------------------------------------------------
Processing Folder: ../r

In [ ]:
import pyiqa
import torch
from PIL import Image

model = pyiqa.create_metric('niqe')
# /home/pearson/Projects/ECE253/Image_process/example_jpg/input/IMG_3583.jpeg
img = Image.open('/home/pearson/Projects/ECE253/Image_process/example_jpg/output_CLAHE/output_clahe_IMG_3583.jpeg')

score = model(img)
print("NIQE =", score.item())

NIQE = 4.862559032140067


In [63]:
import cv2
import numpy as np
from skimage import io
import pyiqa
# from transformers import AutoModelForImageQualityAssessment, AutoImageProcessor
from PIL import Image
import torch
import json


########################################
# 1. Sharpness (Tenengrad)
########################################
def tenengrad(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_64F, 0, 1)
    return np.mean(gx**2 + gy**2)



########################################
# 3. LOE (Lightness Order Error)
########################################
def compute_loe(input_img, enhanced_img, resize_dim=100):
    """
    计算 LOE (Lightness Order Error)
    :param input_img: 原始低光照图像 (BGR)
    :param enhanced_img: 增强后的图像 (BGR)
    :param resize_dim: 为了加速，通常会将图像缩放到较小尺寸 (如 100x100)
    :return: LOE 值 (越小越好)
    """
    # 1. 转为 V 通道 (HSV) 或 L 通道 (LAB) 来提取亮度，或者直接用最大值
    # 论文通常使用 max(R,G,B) 作为亮度层
    L_in = np.max(input_img, axis=2).astype(np.float32)
    L_out = np.max(enhanced_img, axis=2).astype(np.float32)

    # 2. 缩放图像以减少计算量 (这是计算 LOE 的标准操作，否则计算量爆炸)
    L_in = cv2.resize(L_in, (resize_dim, resize_dim))
    L_out = cv2.resize(L_out, (resize_dim, resize_dim))

    # 展平为一维数组
    vec_in = L_in.flatten()
    vec_out = L_out.flatten()
    N = len(vec_in)

    # 3. 利用广播机制计算相对顺序矩阵
    # creating a matrix of comparisons: matrix[i, j] = vec[i] >= vec[j]
    
    # 原图的相对顺序矩阵 (如果 i >= j 则为 True)
    # 使用广播: vec_in[:, None] 是列向量, vec_in[None, :] 是行向量
    order_in = vec_in[:, None] >= vec_in[None, :]
    
    # 增强图的相对顺序矩阵
    order_out = vec_out[:, None] >= vec_out[None, :]

    # 4. 计算异或 (XOR)
    # 如果原图和增强图的顺序关系不一致 (一个True一个False)，异或结果为 True (1)
    difference_matrix = np.logical_xor(order_in, order_out)

    # 5. 计算 LOE
    # 对所有差异求和，然后归一化
    loe_score = np.mean(np.sum(difference_matrix, axis=1))
    
    return loe_score

# 使用示例
# input_img = cv2.imread('low.jpg')
# enhanced_img = cv2.imread('enhanced.jpg')
# score = compute_loe(input_img, enhanced_img)
# print(f"LOE Score: {score}")


# ########################################
# # 4. Deep metric: MUSIQ
# ########################################
# def load_musiq():
#     model = AutoModelForImageQualityAssessment.from_pretrained("google/musiq")
#     processor = AutoImageProcessor.from_pretrained("google/musiq")
#     return model, processor


# def compute_musiq(img_pil, model, processor):
#     inputs = processor(images=img_pil, return_tensors="pt")
#     with torch.no_grad():
#         score = model(**inputs).logits.item()
#     return score


def compute_maniqa(img_pil):
    model = pyiqa.create_metric('maniqa')
    return model(img_pil).item()


########################################
# 5. Main pipeline
########################################
def evaluate(image_path):
    # Load image
    img = cv2.imread(image_path)
    img_pil = Image.open(image_path)

    # pyiqa metrics
    niqe = pyiqa.create_metric('niqe')
    brisque = pyiqa.create_metric('brisque')
    piqe = pyiqa.create_metric('piqe')

    niqe_score = niqe(img_pil).item()
    brisque_score = brisque(img_pil).item()
    piqe_score = piqe(img_pil).item()

    # Classical metrics
    tenengrad_score = tenengrad(img)
    # entropy_score = entropy(img)
    # loe_score = compute_loe(img)

    # # MUSIQ
    maniqa_score = compute_maniqa(img_pil)
    # musiq_model, musiq_processor = load_musiq()
    # musiq_score = compute_musiq(img_pil, musiq_model, musiq_processor)

    result = {
        "NIQE": niqe_score,
        # "BRISQUE": brisque_score,
        # "PIQE": piqe_score,
        "MANIQ": maniqa_score,
        "Tenengrad": tenengrad_score,
    }

    return result


########################################
# Run example
########################################


In [65]:

import argparse
# parser = argparse.ArgumentParser()
# parser.add_argument("--image", type=str, required=True)
# args = parser.parse_args()
img1 = '/home/pearson/Projects/ECE253/Image_process/example_jpg/input/IMG_3580.jpeg'
scores1 = evaluate(img1)

print(json.dumps(scores1, indent=4))


img2 = '/home/pearson/Projects/ECE253/Image_process/example_jpg/output_CLAHE/output_clahe_IMG_3580.jpeg'
scores2 = evaluate(img2)

print(json.dumps(scores2, indent=4))


img3 = '/home/pearson/Projects/ECE253/Image_process/example_jpg/output_ZeroDCE/IMG_3580.jpeg'
scores3 = evaluate(img3)

print(json.dumps(scores3, indent=4))



Loading pretrained model MANIQA from /home/pearson/.cache/torch/hub/pyiqa/ckpt_koniq10k.pt
{
    "NIQE": 5.12974739878657,
    "MANIQ": 0.25075048208236694,
    "Tenengrad": 823.0470265652557
}
Loading pretrained model MANIQA from /home/pearson/.cache/torch/hub/pyiqa/ckpt_koniq10k.pt
{
    "NIQE": 3.869808225234333,
    "MANIQ": 0.20987872779369354,
    "Tenengrad": 3450.0166497057926
}
Loading pretrained model MANIQA from /home/pearson/.cache/torch/hub/pyiqa/ckpt_koniq10k.pt
{
    "NIQE": 4.342191332573618,
    "MANIQ": 0.2231995016336441,
    "Tenengrad": 4722.289415660169
}


In [66]:
# input_img = cv2.imread('low.jpg')
# enhanced_img = cv2.imread('enhanced.jpg')
input_img_path = '/home/pearson/Projects/ECE253/Image_process/example_jpg/input/IMG_3583.jpeg'
enhanced_img1_path = '/home/pearson/Projects/ECE253/Image_process/example_jpg/output_CLAHE/output_clahe_IMG_3583.jpeg'
enhanced_img2_path = '/home/pearson/Projects/ECE253/Image_process/example_jpg/output_ZeroDCE/IMG_3583.jpeg'

input_img = cv2.imread(input_img_path)
enhanced_img1 = cv2.imread(enhanced_img1_path)
enhanced_img2 = cv2.imread(enhanced_img2_path)
score1 = compute_loe(input_img, enhanced_img1)
score2 = compute_loe(input_img, enhanced_img2)
print(score1)
print(score2)

816.9036
5399.7899
